In [46]:
!ls runs |wc -l

     133


In [174]:
import os
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import clipboard
import base64
from io import BytesIO
from PIL import Image


In [175]:
main_path = './runs/'

In [176]:
def get_event_files(main_path):
    """Return a list of event files under the given directory"""
    all_files = []
    for root, _, filenames in os.walk(main_path):
        for filename in filenames:
            if "events.out.tfevents" in filename:
                all_files.append(str(Path(root) / Path(filename)))
    return all_files

In [177]:
def process_event_acc(event_acc):
    """Process the EventAccumulator and return a dictionary of tag values"""
    all_tags = event_acc.Tags()
    temp_dict = {}
    for tag in all_tags.keys():
        if tag == 'scalars':
            for subtag in all_tags[tag]:
                temp_dict[subtag] = [tag[-1] for tag in event_acc.Scalars(tag=subtag)][-1]
        if tag == 'tensors':
            for subtag in all_tags[tag]:
                temp_dict[subtag.replace('/text_summary', "")] = [tag[-1] for tag in event_acc.Tensors(tag=subtag)][0].string_val[0].decode('ascii')
        if tag == 'images':
            for subtag in all_tags[tag]:
                temp_dict[subtag] = Image.open(BytesIO(event_acc.Images(subtag)[1].encoded_image_string))
    return temp_dict

In [178]:
def process_runs(main_path):
    all_files = get_event_files(main_path=main_path)
    all_dict = {}
    for files in tqdm(all_files, total = len(all_files)):
        event_acc = EventAccumulator(files)
        event_acc.Reload()
        temp_dict = process_event_acc(event_acc)
        all_dict[files] = temp_dict
    return pd.DataFrame.from_records(all_dict).T.reset_index()

In [179]:
combined_df = process_runs(main_path=main_path)
combined_df = combined_df[(~pd.isnull(combined_df['experiment_name'])) & (~pd.isnull(combined_df['Loss/Val']))]
combined_df.head()

100%|██████████| 441/441 [00:00<00:00, 696.71it/s]


,index,proxy_step,Loss/Train,Acc/Train,Loss/Val,Acc/Val,experiment_name,ds_name,image_size,batch_size,...,ds_path,name_fn,global_run_count,label_map,rev_label_map,num_classes,original_images,converted_proxy,Number_Chosen,save_path
0,"runs/asl_blend_test+08032023_16:30:00_ps-[10, ...",0.0,0.478703,0.85625,0.115334,0.96525,blend_test,asl,224,32,...,/mnt/e/Datasets/asl/asl_alphabet_train/asl_alp...,<function get_parent_name at 0x7f817d4d55a0>,0,"{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F...","{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': ...",29,NaN,NaN,NaN,NaN
1,"runs/asl_blend_test+08032023_16:30:00_ps-[10, ...",1.0,0.510605,0.84025,0.14249,0.95675,blend_test,asl,224,32,...,/mnt/e/Datasets/asl/asl_alphabet_train/asl_alp...,<function get_parent_name at 0x7f817d4d55a0>,10,"{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F...","{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': ...",29,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,10.0,/mnt/e/CODE/Github/improving_robotics_datasets...
2,"runs/asl_blend_test+08032023_16:30:00_ps-[10, ...",0.0,0.367487,0.88625,0.036445,0.98925,blend_test,asl,224,32,...,/mnt/e/Datasets/asl/asl_alphabet_train/asl_alp...,<function get_parent_name at 0x7f817d4d55a0>,11,"{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F...","{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': ...",29,NaN,NaN,NaN,/mnt/e/CODE/Github/improving_robotics_datasets...
8,"runs/asl_blend_test+08032023_16:38:51_ps-[10, ...",0.0,0.493784,0.85425,0.111464,0.96925,blend_test,asl,224,32,...,/mnt/e/Datasets/asl/asl_alphabet_train/asl_alp...,<function get_parent_name at 0x7f6bf04815a0>,0,"{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F...","{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': ...",29,NaN,NaN,NaN,NaN
9,"runs/asl_blend_test+08032023_16:38:51_ps-[10, ...",1.0,0.516414,0.84,0.100197,0.967,blend_test,asl,224,32,...,/mnt/e/Datasets/asl/asl_alphabet_train/asl_alp...,<function get_parent_name at 0x7f6bf04815a0>,10,"{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F...","{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': ...",29,<PIL.PngImagePlugin.PngImageFile image mode=RG...,<PIL.PngImagePlugin.PngImageFile image mode=RG...,26.0,/mnt/e/CODE/Github/improving_robotics_datasets...


In [180]:
combined_df.shape

(142, 36)

In [181]:
combined_df["global_run_count"].value_counts()

0     88
10    27
11    27
Name: global_run_count, dtype: int64

In [182]:
combined_df[combined_df["global_run_count"]!= "0"]["proxy_steps"].value_counts()

[10, 'p', 10]    54
Name: proxy_steps, dtype: int64

# Image code

In [183]:
filtered_df = combined_df[(~pd.isnull(combined_df['converted_proxy'])) & (~pd.isnull(combined_df['original_images']))]

In [184]:
filtered_df.iloc[0].original_images

In [185]:
filtered_df.iloc[0].converted_proxy

# Preprocess

In [157]:
def check_proxy(string): return "p" in string

In [158]:
def calc_stats(values):
    return f"min: {values.min()} \nmax: {values.max()} \navg: {values.mean()}"

In [159]:
def convert_float(df, cols, totype= float):
    for col in cols:
        df[col] = df[col].astype(totype)

In [160]:
combined_df = combined_df.fillna(0)
# col to check for proxy
# combined_df["has_proxy"] = combined_df["proxy_steps"].apply(check_proxy)
# Fix naming
combined_df = combined_df.rename(columns={"Acc/Val":"accuracy", "proxy_steps":"step_schedule"})
# Fix types
convert_float(combined_df, ["change_subset_attention", "proxy_threshold", "accuracy"], float)
convert_float(combined_df, ["global_run_count"], int)
convert_float(combined_df, ["transfer_imagenet"], bool)

# ignore failed runs
combined_df = combined_df[combined_df["global_run_count"]!=0]


# Grouped Results

In [161]:
def return_grouped_results(df, group_cols ,filter = None, index_cols = (["ds_name", ("accuracy")]), print_latex = False):
    if filter != None:
        df = df.reset_index()
        for key in filter.keys():
            df = df[df[key] == filter[key]]
    final_df = pd.DataFrame(df.groupby(group_cols, as_index=True).mean(numeric_only = True)["accuracy"]).sort_values(index_cols, ascending=False)
    if print_latex == True:
        clipboard.copy(final_df.to_latex())

    return final_df



In [162]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule"], filter = {"transfer_imagenet":True , "ds_name":"asl"})

,,,,,accuracy
ds_name,pixel_replacement_method,transfer_imagenet,gradient_method,step_schedule,
asl,blended,True,gradcamplusplus,"[10, 'p', 10]",0.966371


In [163]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "step_schedule", "global_run_count"])

accuracy
ds_name    pixel_replacement_method transfer_imagenet gradient_method step_schedule global_run_count          
imagenette blended                  True              gradcamplusplus [10, 'p', 10] 11                0.710021
                                                                                    10                0.662625
asl        blended                  True              gradcamplusplus [10, 'p', 10] 11                0.969076
                                                                                    10                0.963667

In [164]:
return_grouped_results(combined_df, ["ds_name", "pixel_replacement_method", "transfer_imagenet", "gradient_method", "change_subset_attention", "step_schedule"])

accuracy
ds_name    pixel_replacement_method transfer_imagenet gradient_method change_subset_attention step_schedule          
imagenette blended                  True              gradcamplusplus 0.8                     [10, 'p', 10]  0.686323
asl        blended                  True              gradcamplusplus 0.3                     [10, 'p', 10]  0.973000
                                                                      0.8                     [10, 'p', 10]  0.963057

## Parameter elimination

In [428]:
def check_wrt_accuracy(has_proxy, condition, head = None):
    grouped_vals = has_proxy.groupby([condition, "ds_name", "transfer_imagenet"]).mean(numeric_only = True)["accuracy"].reset_index().sort_values(by = ["accuracy"], ascending=False)
    print("Transfer")
    if head != None:
        print(grouped_vals[grouped_vals["transfer_imagenet"] == True].head(head).to_markdown())
        print("No Transfer")
        print(grouped_vals[grouped_vals["transfer_imagenet"] == False].head(head).to_markdown())
    else:
        print(grouped_vals[grouped_vals["transfer_imagenet"] == True].to_markdown())
        print("No Transfer")
        print(grouped_vals[grouped_vals["transfer_imagenet"] == False].to_markdown())

In [429]:
check_wrt_accuracy(combined_df, "clear_every_step")

Transfer
|    | clear_every_step   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:-------------------|:-----------|:--------------------|-----------:|
|  0 | False              | asl        | True                |   0.965529 |
|  2 | True               | asl        | True                |   0.926681 |
|  1 | False              | imagenette | True                |   0.816    |
|  3 | True               | imagenette | True                |   0.66642  |
No Transfer
| clear_every_step   | ds_name   | transfer_imagenet   | accuracy   |
|--------------------|-----------|---------------------|------------|


In [430]:
check_wrt_accuracy(combined_df, "change_subset_attention")

Transfer
|    |   change_subset_attention | ds_name    | transfer_imagenet   |   accuracy |
|---:|--------------------------:|:-----------|:--------------------|-----------:|
|  4 |                       0.8 | asl        | True                |   0.957823 |
|  0 |                       0.3 | asl        | True                |   0.935525 |
|  2 |                       0.5 | asl        | True                |   0.911833 |
|  1 |                       0.3 | imagenette | True                |   0.771125 |
|  3 |                       0.5 | imagenette | True                |   0.769875 |
|  5 |                       0.8 | imagenette | True                |   0.663041 |
No Transfer
| change_subset_attention   | ds_name   | transfer_imagenet   | accuracy   |
|---------------------------|-----------|---------------------|------------|


In [432]:
check_wrt_accuracy(combined_df, "gradient_method")

Transfer
|    | gradient_method   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:------------------|:-----------|:--------------------|-----------:|
|  0 | gradcamplusplus   | asl        | True                |   0.938634 |
|  1 | gradcamplusplus   | imagenette | True                |   0.674157 |
No Transfer
| gradient_method   | ds_name   | transfer_imagenet   | accuracy   |
|-------------------|-----------|---------------------|------------|


In [433]:
check_wrt_accuracy(combined_df, "pixel_replacement_method")

Transfer
|    | pixel_replacement_method   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:---------------------------|:-----------|:--------------------|-----------:|
|  0 | blended                    | asl        | True                |   0.938634 |
|  1 | blended                    | imagenette | True                |   0.674157 |
No Transfer
| pixel_replacement_method   | ds_name   | transfer_imagenet   | accuracy   |
|----------------------------|-----------|---------------------|------------|


In [434]:
check_wrt_accuracy(combined_df, "proxy_threshold", head= 4)

Transfer
|    |   proxy_threshold | ds_name   | transfer_imagenet   |   accuracy |
|---:|------------------:|:----------|:--------------------|-----------:|
| 77 |          0.92068  | asl       | True                |    0.98875 |
| 84 |          0.94807  | asl       | True                |    0.9825  |
| 71 |          0.898883 | asl       | True                |    0.98    |
| 75 |          0.913456 | asl       | True                |    0.975   |
No Transfer
| proxy_threshold   | ds_name   | transfer_imagenet   | accuracy   |
|-------------------|-----------|---------------------|------------|


In [435]:

check_wrt_accuracy(combined_df, "subset_images")

Transfer
|    |   subset_images | ds_name    | transfer_imagenet   |   accuracy |
|---:|----------------:|:-----------|:--------------------|-----------:|
|  0 |            8000 | asl        | True                |   0.938634 |
|  1 |            8000 | imagenette | True                |   0.674157 |
No Transfer
| subset_images   | ds_name   | transfer_imagenet   | accuracy   |
|-----------------|-----------|---------------------|------------|


In [437]:

check_wrt_accuracy(combined_df, "step_schedule")

Transfer
|    | step_schedule   | ds_name    | transfer_imagenet   |   accuracy |
|---:|:----------------|:-----------|:--------------------|-----------:|
|  4 | [21]            | asl        | True                |   0.976833 |
|  0 | [10, 'p', 10]   | asl        | True                |   0.962566 |
|  2 | [20, 'p', 10]   | asl        | True                |   0.88725  |
|  6 | [31]            | asl        | True                |   0.87725  |
|  5 | [21]            | imagenette | True                |   0.809167 |
|  1 | [10, 'p', 10]   | imagenette | True                |   0.669798 |
|  7 | [31]            | imagenette | True                |   0.62725  |
|  3 | [20, 'p', 10]   | imagenette | True                |   0.602167 |
No Transfer
| step_schedule   | ds_name   | transfer_imagenet   | accuracy   |
|-----------------|-----------|---------------------|------------|
